In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plot
from tensorflow.examples.tutorials.mnist import input_data 

#从网上下载数据
mnist_data = input_data.read_data_sets("MNIST_data",one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
print (mnist_data.train.images.shape)

(55000, 784)


In [4]:
#程序会自动对数据集进行解压缩,
#2 对数据设置每个批次的大小
batch_size = 100
n_batch = int(mnist_data.train.num_examples/batch_size)

# 3 定义两个placeholder，用来承载数据，因为每个图片，都是一个784维的数据，所以我们的x是784列；
#   因为要把图片识别为0-9的10个数字，也就是有10个标签，所以y是10列
x = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,10])

epoch = 0 


def weight_variable(shape):
    # 使用正态分布随机生成初始权重 标准方差为0.1
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    # 初始偏移量设为非零，避免死神经元
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

    
def conv2d(x, W):
    # 卷积不改变输入的shape
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='VALID')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                              strides=[1, 2, 2, 1], padding='SAME')

In [5]:
# 把输入变换成一个4d的张量，第二三个对应的是图片的长和宽，第四个参数对应的颜色
x_image = tf.reshape(x, [-1, 28, 28, 1])

In [6]:
#第1层：卷积层，滤波器的尺寸为5×5，深度为6,不使用全0补充，步长为1。
 
W_conv1 = weight_variable([5,5, 1, 6])
# 偏差的shape应该和输出的shape一致，所以也是6
b_conv1 = bias_variable([6])
# 对数据做卷积操作
#尺寸变化：32×32×1->28×28×6
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

# 第2层，池化层
#池化层的尺寸为2×2，使用全0补充，步长为2。
    #尺寸变化：28×28×6->14×14×6
h_pool1 = max_pool_2x2(h_conv1)

#第3层：卷积层
#滤波器的尺寸为5×5，深度为16,不使用全0补充，步长为1。
#尺寸变化：14×14×6->10×10×16
W_conv2 = weight_variable([5, 5, 6, 16])
b_conv2 = bias_variable([16])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)

# 第4层：池化层
#尺寸变化：10×10×6->5×5×16
h_pool2 = max_pool_2x2(h_conv2)

# 第5层，全连接层。
#本层的输入矩阵大小为5×5×16，在LeNet-5模型的论文中将这一层称为卷积层，
#但是因为过滤器的大小就是5×5，所以和全连接层没有区别，这里直接看成全连接层
# 把数据拉平变成5×5×16=400的向量
pool_shape = h_pool2.get_shape().as_list()
nodes = pool_shape[1]*pool_shape[2]*pool_shape[3]
reshaped = tf.reshape(h_pool2,[-1,nodes])

W_fc1 = weight_variable([nodes, 120])
b_fc1 = bias_variable([120])
h_fc1 = tf.nn.relu(tf.matmul(reshaped, W_fc1) + b_fc1)

# 第6层，全连接层
# 120->84的全连接
W_fc2 = weight_variable([120, 84])
b_fc2 = bias_variable([84])
y_fc2 = tf.matmul(h_fc1, W_fc2) + b_fc2

#第7层：全连接层，84->10的全连接
#得到最终结果
W_fc3 = weight_variable([84, 10])
b_fc3 = bias_variable([10])
y_p = tf.matmul(y_fc2, W_fc3) + b_fc3

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
# 创建损失函数,以交叉熵的平均值为衡量
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y, logits =y_p ))

# 用梯度下降法优化参数

train_step_1 = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)


# 计算准确度
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_p,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))


global_step = tf.Variable(0,name = 'global_step', trainable = False)

saver = tf.train.Saver()

# 初始化变量
init = tf.global_variables_initializer()

In [9]:
with tf.Session() as sess:
    sess.run(init)
#     saver.restore(sess,"model.ckpt-7")
    for epoch in range(1,20):
        for batch in range(n_batch):
            
            batch_x,batch_y = mnist_data.train.next_batch(batch_size)
            sess.run(train_step_1,feed_dict = {x:batch_x,y:batch_y})
        
        accuracy_n = sess.run(accuracy,feed_dict={ x: mnist_data.test.images, y: mnist_data.test.labels})
        
        print ("第" + str(epoch+1)+"轮，准确度为："+str(accuracy_n))
        
        global_step.assign(epoch).eval()
        
        saver.save(sess,"model-LeNet5.ckpt",global_step = global_step)

第2轮，准确度为：0.9709
第3轮，准确度为：0.9777
第4轮，准确度为：0.9826
第5轮，准确度为：0.9832
第6轮，准确度为：0.9863
第7轮，准确度为：0.9876
Instructions for updating:
Use standard file APIs to delete files with this prefix.
第8轮，准确度为：0.9878
第9轮，准确度为：0.989
第10轮，准确度为：0.9894
第11轮，准确度为：0.9893
第12轮，准确度为：0.9845
第13轮，准确度为：0.9902
第14轮，准确度为：0.9887
第15轮，准确度为：0.9873
第16轮，准确度为：0.9883
第17轮，准确度为：0.9891
第18轮，准确度为：0.9864
第19轮，准确度为：0.9879
第20轮，准确度为：0.9892
